In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is the PRIMARY learning material!")

---

## Part 1: GPU Cache Hierarchy

### Memory Hierarchy

```
GPU Memory Hierarchy:

┌─────────────────────────────────────────────────────────────┐
│                     Registers (per thread)                   │
│                     ~1 cycle, 255 max                        │
└─────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────┐
│              Shared Memory / L1 Cache (per SM)               │
│              ~5-30 cycles, 48-228KB configurable             │
└─────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────┐
│                    L2 Cache (shared)                         │
│                    ~100-200 cycles, 1.5-80MB                 │
└─────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────┐
│                 Global Memory (DRAM)                         │
│                 ~400-800 cycles, 8-80GB                      │
└─────────────────────────────────────────────────────────────┘
```

### L1 Cache Details

```
L1 Cache (per SM):
  - Unified with shared memory (128-228KB total)
  - Configurable split: more shared OR more L1
  - Cache line: 128 bytes
  - Write-through (writes go to L2)
  - Automatic caching of global loads

Configuration options (cudaFuncSetCacheConfig):
  cudaFuncCachePreferNone     Default
  cudaFuncCachePreferShared   Prefer shared memory
  cudaFuncCachePreferL1       Prefer L1 cache
  cudaFuncCachePreferEqual    Equal split
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile cache_config.cu
// cache_config.cu - Configure L1/shared memory split
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void memoryIntensiveKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Read many times from different locations
        float sum = 0;
        for (int i = 0; i < 16; i++) {
            int offset = (idx + i * 1024) % n;
            sum += data[offset];
        }
        data[idx] = sum;
    }
}

int main() {
    // Query current cache config
    cudaFuncCache currentConfig;
    cudaDeviceGetCacheConfig(&currentConfig);
    printf("Current config: %d\n", currentConfig);
    
    // Set cache preference for kernel
    cudaFuncSetCacheConfig(memoryIntensiveKernel, cudaFuncCachePreferL1);
    
    // For kernels that use shared memory heavily:
    // cudaFuncSetCacheConfig(kernel, cudaFuncCachePreferShared);
    
    // Query device L1/shared config
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Shared memory per block: %zu KB\n", prop.sharedMemPerBlock / 1024);
    printf("Shared memory per SM: %zu KB\n", prop.sharedMemPerMultiprocessor / 1024);
    printf("L2 cache size: %d KB\n", prop.l2CacheSize / 1024);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o cache_config cache_config.cu
!./cache_config

### 🔶 Python/Numba (Optional - Quick Testing)

In [ ]:
# Query cache information
device = cuda.get_current_device()

print("GPU Cache Information")
print("=" * 50)
print(f"Device: {device.name}")
print(f"Max shared memory per block: {device.MAX_SHARED_MEMORY_PER_BLOCK / 1024:.0f} KB")

# Get more detailed info via context
ctx = cuda.current_context()
try:
    print(f"\nNote: L1/L2 cache sizes vary by GPU architecture")
    print(f"Typical L1: 48-128 KB per SM")
    print(f"Typical L2: 1.5-40 MB shared")
except:
    pass

---

## Part 2: Memory Coalescing Deep Dive

In [ ]:
def explain_coalescing():
    """Deep dive into memory coalescing."""
    print("Memory Coalescing")
    print("=" * 60)
    print()
    print("Definition: Combining multiple memory requests into fewer")
    print("            transactions at cache-line granularity (128 bytes)")
    print()
    print("COALESCED ACCESS (GOOD):")
    print("  Warp threads 0-31 access addresses 0-124 (consecutive floats)")
    print("  → 1 cache line transaction (128 bytes = 32 floats)")
    print()
    print("  Thread 0:  addr 0")
    print("  Thread 1:  addr 4")
    print("  Thread 2:  addr 8")
    print("  ...")
    print("  Thread 31: addr 124")
    print("  ────────────────────")
    print("  Result: ONE 128-byte transaction")
    print()
    print("STRIDED ACCESS (BAD):")
    print("  Warp threads access every 16th element (stride = 64 bytes)")
    print("  → Multiple cache line transactions")
    print()
    print("  Thread 0:  addr 0    → cache line 0")
    print("  Thread 1:  addr 64   → cache line 0")
    print("  Thread 2:  addr 128  → cache line 1")
    print("  Thread 3:  addr 192  → cache line 1")
    print("  ...")
    print("  Result: 16+ cache line transactions!")

explain_coalescing()

In [ ]:
# Demonstrate coalescing impact

@cuda.jit
def coalesced_access(data, result, stride):
    """Stride-1 access (coalesced)."""
    idx = cuda.grid(1)
    if idx < result.size:
        result[idx] = data[idx]

@cuda.jit
def strided_access(data, result, stride):
    """Strided access (not coalesced)."""
    idx = cuda.grid(1)
    if idx < result.size:
        # Strided read - bad for coalescing
        src_idx = (idx * stride) % data.size
        result[idx] = data[src_idx]

In [ ]:
def benchmark_coalescing(n=1_000_000):
    """Benchmark coalesced vs strided access."""
    data = np.random.rand(n * 32).astype(np.float32)  # Extra space for strides
    result = np.zeros(n, dtype=np.float32)
    
    d_data = cuda.to_device(data)
    d_result = cuda.to_device(result)
    
    block = 256
    grid = (n + block - 1) // block
    
    print(f"Coalescing Benchmark ({n:,} elements)")
    print("=" * 50)
    
    for stride in [1, 2, 4, 8, 16, 32]:
        # Warmup
        strided_access[grid, block](d_data, d_result, stride)
        cuda.synchronize()
        
        # Benchmark
        iterations = 100
        start = time.perf_counter()
        for _ in range(iterations):
            strided_access[grid, block](d_data, d_result, stride)
        cuda.synchronize()
        elapsed = (time.perf_counter() - start) / iterations * 1000
        
        # Calculate bandwidth
        bytes_moved = n * 4 * 2  # read + write
        bandwidth = bytes_moved / (elapsed / 1000) / 1e9
        
        print(f"Stride {stride:2d}: {elapsed:.3f} ms, {bandwidth:.1f} GB/s")

benchmark_coalescing()

---

## Part 3: L2 Cache Optimization

In [ ]:
def l2_cache_strategies():
    """Strategies for L2 cache optimization."""
    print("L2 Cache Optimization Strategies")
    print("=" * 60)
    print()
    print("1. DATA LOCALITY")
    print("   Keep working set smaller than L2 cache")
    print("   Typical L2: 1.5-40 MB")
    print()
    print("2. PERSISTENCE (Ampere+)")
    print("   cudaAccessPropertyPersisting: keep in L2")
    print("   cudaAccessPropertyStreaming: don't cache")
    print()
    print("3. CACHE PARTITIONING (Ampere+)")
    print("   Reserve portion of L2 for specific data")
    print("   cudaDeviceSetLimit(cudaLimitPersistingL2CacheSize, bytes)")
    print()
    print("4. ACCESS PATTERNS")
    print("   Sequential: Best cache utilization")
    print("   Random: Poor cache utilization")
    print("   Blocked/Tiled: Good cache reuse")

l2_cache_strategies()

### 🔷 CUDA C++ L2 Cache Control (Ampere+)

In [ ]:
%%writefile l2_cache.cu
// l2_cache.cu - L2 cache persistence (Ampere+)
#include <stdio.h>
#include <cuda_runtime.h>

int main() {
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    
    printf("Device: %s\n", prop.name);
    printf("L2 cache size: %d MB\n", prop.l2CacheSize / (1024 * 1024));
    printf("Compute capability: %d.%d\n", prop.major, prop.minor);
    
    // On Ampere+, you can control L2 persistence
    if (prop.major >= 8) {
        printf("\nAmpere+ detected - L2 persistence available\n");
        
        // Reserve some L2 cache for persistent data
        size_t persistingL2 = prop.l2CacheSize / 2;  // Reserve half
        cudaDeviceSetLimit(cudaLimitPersistingL2CacheSize, persistingL2);
        printf("Reserved %zu KB for L2 persistence\n", persistingL2 / 1024);
        
        // Set access policy for a memory range
        float* d_persistent;
        size_t size = 1024 * 1024;  // 1MB
        cudaMalloc(&d_persistent, size);
        
        cudaStreamAttrValue attr;
        attr.accessPolicyWindow.base_ptr = d_persistent;
        attr.accessPolicyWindow.num_bytes = size;
        attr.accessPolicyWindow.hitProp = cudaAccessPropertyPersisting;
        attr.accessPolicyWindow.missProp = cudaAccessPropertyStreaming;
        attr.accessPolicyWindow.hitRatio = 1.0f;
        
        cudaStream_t stream;
        cudaStreamCreate(&stream);
        cudaStreamSetAttribute(stream, cudaStreamAttributeAccessPolicyWindow, &attr);
        
        printf("Set up L2 persistence policy for 1MB buffer\n");
        
        // Now kernels on this stream will try to keep d_persistent in L2
        
        cudaStreamDestroy(stream);
        cudaFree(d_persistent);
    } else {
        printf("\nPre-Ampere GPU - L2 persistence not available\n");
        printf("But automatic L2 caching still works!\n");
    }
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o l2_cache l2_cache.cu
!./l2_cache

---

## Part 4: Texture and Constant Memory

In [ ]:
def special_caches():
    """Explain texture and constant memory caches."""
    print("Special GPU Caches")
    print("=" * 60)
    print()
    print("CONSTANT MEMORY")
    print("  Size: 64 KB total, 8 KB cache per SM")
    print("  Access: Broadcast to all threads in warp")
    print("  Best for: Same value read by all threads")
    print("  Declare: __constant__ float data[1024];")
    print()
    print("TEXTURE MEMORY")
    print("  Cache: ~48 KB per SM")
    print("  Access: Optimized for 2D spatial locality")
    print("  Features:")
    print("    - Hardware interpolation")
    print("    - Boundary handling (clamp/wrap)")
    print("    - Normalized coordinates")
    print("  Best for: Image processing, lookup tables")
    print()
    print("READ-ONLY CACHE (via __ldg)")
    print("  Uses texture cache for global loads")
    print("  float x = __ldg(&data[idx]);")
    print("  Automatic with 'const __restrict__' pointers")

special_caches()

### 🔷 CUDA C++ Constant Memory

In [ ]:
%%writefile constant_memory.cu
// constant_memory.cu - Using constant memory
#include <stdio.h>
#include <cuda_runtime.h>

// Declare constant memory (at file scope)
__constant__ float coefficients[256];

__global__ void applyCoefficients(float* data, float* result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        float x = data[idx];
        int coef_idx = idx % 256;
        
        // All threads in warp likely read same coefficient
        // → Constant cache broadcasts efficiently
        result[idx] = x * coefficients[coef_idx];
    }
}

int main() {
    int n = 1 << 20;  // 1M elements
    
    // Copy to constant memory
    float h_coefs[256];
    for (int i = 0; i < 256; i++) {
        h_coefs[i] = i * 0.1f;
    }
    cudaMemcpyToSymbol(coefficients, h_coefs, sizeof(h_coefs));
    
    // Allocate and initialize data
    float *h_data = (float*)malloc(n * sizeof(float));
    float *h_result = (float*)malloc(n * sizeof(float));
    for (int i = 0; i < n; i++) h_data[i] = 1.0f;
    
    float *d_data, *d_result;
    cudaMalloc(&d_data, n * sizeof(float));
    cudaMalloc(&d_result, n * sizeof(float));
    cudaMemcpy(d_data, h_data, n * sizeof(float), cudaMemcpyHostToDevice);
    
    // Launch kernel
    applyCoefficients<<<(n+255)/256, 256>>>(d_data, d_result, n);
    
    cudaMemcpy(h_result, d_result, n * sizeof(float), cudaMemcpyDeviceToHost);
    
    printf("result[0] = %f (expected: %f)\n", h_result[0], h_coefs[0]);
    printf("result[1] = %f (expected: %f)\n", h_result[1], h_coefs[1]);
    printf("result[255] = %f (expected: %f)\n", h_result[255], h_coefs[255]);
    
    free(h_data);
    free(h_result);
    cudaFree(d_data);
    cudaFree(d_result);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o constant_memory constant_memory.cu
!./constant_memory

### 🔷 Using __ldg for Read-Only Data

In [ ]:
%%writefile ldg_readonly.cu
// ldg_readonly.cu - Using __ldg for read-only data
#include <stdio.h>
#include <cuda_runtime.h>

// Option 1: Explicit __ldg intrinsic
__global__ void withLdg(const float* data, float* result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Uses texture cache path
        float x = __ldg(&data[idx]);
        result[idx] = x * x;
    }
}

// Option 2: const __restrict__ (compiler may use __ldg)
__global__ void withRestrict(const float* __restrict__ data, 
                             float* __restrict__ result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        // Compiler knows data is read-only and non-aliasing
        result[idx] = data[idx] * data[idx];
    }
}

int main() {
    int n = 1 << 20;
    
    float *h_data = (float*)malloc(n * sizeof(float));
    float *h_result = (float*)malloc(n * sizeof(float));
    for (int i = 0; i < n; i++) h_data[i] = (float)i;
    
    float *d_data, *d_result;
    cudaMalloc(&d_data, n * sizeof(float));
    cudaMalloc(&d_result, n * sizeof(float));
    cudaMemcpy(d_data, h_data, n * sizeof(float), cudaMemcpyHostToDevice);
    
    // Test both kernels
    withLdg<<<(n+255)/256, 256>>>(d_data, d_result, n);
    cudaMemcpy(h_result, d_result, n * sizeof(float), cudaMemcpyDeviceToHost);
    printf("withLdg: result[2] = %f (expected: %f)\n", h_result[2], 4.0f);
    
    withRestrict<<<(n+255)/256, 256>>>(d_data, d_result, n);
    cudaMemcpy(h_result, d_result, n * sizeof(float), cudaMemcpyDeviceToHost);
    printf("withRestrict: result[2] = %f (expected: %f)\n", h_result[2], 4.0f);
    
    free(h_data);
    free(h_result);
    cudaFree(d_data);
    cudaFree(d_result);
    
    return 0;
}

### 🔶 Python/Numba (Optional - Quick Testing)

---

## Part 5: Cache-Aware Algorithm Design

In [ ]:
# Example: Cache-friendly vs cache-unfriendly matrix access

@cuda.jit
def row_major_sum(matrix, result, rows, cols):
    """Row-major traversal - cache friendly."""
    idx = cuda.grid(1)
    if idx < rows:
        total = 0.0
        for j in range(cols):
            total += matrix[idx, j]  # Sequential in memory
        result[idx] = total

@cuda.jit
def col_major_sum(matrix, result, rows, cols):
    """Column-major traversal - cache unfriendly for row-major layout."""
    idx = cuda.grid(1)
    if idx < cols:
        total = 0.0
        for i in range(rows):
            total += matrix[i, idx]  # Strided in memory
        result[idx] = total

In [ ]:
def benchmark_matrix_access(rows=1024, cols=1024):
    """Compare row vs column traversal."""
    matrix = np.random.rand(rows, cols).astype(np.float32)
    result_row = np.zeros(rows, dtype=np.float32)
    result_col = np.zeros(cols, dtype=np.float32)
    
    d_matrix = cuda.to_device(matrix)
    d_result_row = cuda.to_device(result_row)
    d_result_col = cuda.to_device(result_col)
    
    block = 256
    grid_row = (rows + block - 1) // block
    grid_col = (cols + block - 1) // block
    
    # Warmup
    row_major_sum[grid_row, block](d_matrix, d_result_row, rows, cols)
    col_major_sum[grid_col, block](d_matrix, d_result_col, rows, cols)
    cuda.synchronize()
    
    # Benchmark row-major
    iterations = 50
    start = time.perf_counter()
    for _ in range(iterations):
        row_major_sum[grid_row, block](d_matrix, d_result_row, rows, cols)
    cuda.synchronize()
    row_time = (time.perf_counter() - start) / iterations * 1000
    
    # Benchmark column-major
    start = time.perf_counter()
    for _ in range(iterations):
        col_major_sum[grid_col, block](d_matrix, d_result_col, rows, cols)
    cuda.synchronize()
    col_time = (time.perf_counter() - start) / iterations * 1000
    
    print(f"Matrix Access Pattern Comparison ({rows}×{cols})")
    print("=" * 50)
    print(f"Row-major (cache-friendly):   {row_time:.3f} ms")
    print(f"Column-major (cache-unfriendly): {col_time:.3f} ms")
    print(f"Ratio: {col_time/row_time:.2f}x slower")

benchmark_matrix_access()

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile cache_exercises.cu
#include <cuda_runtime.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            printf("CUDA Error: %s at line %d\n", cudaGetErrorString(err), __LINE__); \
            exit(1); \
        } \
    } while(0)

// ============================================================
// Exercise 1: Cache Line Utilization Analysis
// ============================================================

// Coalesced access (stride 1) - good cache utilization
__global__ void coalescedAccess(const float* data, float* result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] = data[idx] * 2.0f;
    }
}

// Strided access - poor cache utilization
__global__ void stridedAccess(const float* data, float* result, int n, int stride) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int dataIdx = idx * stride;
    if (dataIdx < n) {
        result[idx] = data[dataIdx] * 2.0f;
    }
}

// ============================================================
// Exercise 2: L1 vs L2 Cache Behavior
// ============================================================

// Small working set - fits in L1
__global__ void l1CacheHit(const float* data, float* result, int workingSetSize, int iters) {
    int tid = threadIdx.x;
    int idx = tid % workingSetSize;
    
    float sum = 0.0f;
    for (int i = 0; i < iters; i++) {
        sum += data[idx];  // Same small set repeatedly - L1 cache hits
    }
    result[tid] = sum;
}

// Large working set - spills to L2
__global__ void l2CacheMiss(const float* data, float* result, int dataSize, int iters) {
    int tid = threadIdx.x;
    
    float sum = 0.0f;
    for (int i = 0; i < iters; i++) {
        int idx = (tid * 1024 + i * 32) % dataSize;  // Random-ish access
        sum += data[idx];
    }
    result[tid] = sum;
}

// ============================================================
// Exercise 3: Read-Only Cache (__ldg)
// ============================================================

// Normal load
__global__ void normalLoad(const float* __restrict__ data, float* result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] = data[idx] * 2.0f + data[(idx + 1) % n] * 0.5f;
    }
}

// Using __ldg (read-only cache path)
__global__ void ldgLoad(const float* __restrict__ data, float* result, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        result[idx] = __ldg(&data[idx]) * 2.0f + __ldg(&data[(idx + 1) % n]) * 0.5f;
    }
}

// ============================================================
// Benchmarks
// ============================================================

void benchmarkCoalescing() {
    printf("=== Exercise 1: Cache Line Utilization ===\n");
    
    const int n = 1 << 24;  // 16M elements
    const int numAccessed = 1 << 20;  // 1M actual accesses
    
    float *d_data, *d_result;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_result, n * sizeof(float)));
    
    int blockSize = 256;
    int gridSize = (numAccessed + blockSize - 1) / blockSize;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    int strides[] = {1, 2, 4, 8, 16, 32};
    int numStrides = sizeof(strides) / sizeof(strides[0]);
    
    printf("%-10s %-15s %-15s %-15s\n", "Stride", "Time (ms)", "Bandwidth", "Efficiency");
    printf("----------------------------------------------------------\n");
    
    for (int s = 0; s < numStrides; s++) {
        int stride = strides[s];
        
        // Warmup
        stridedAccess<<<gridSize, blockSize>>>(d_data, d_result, n, stride);
        cudaDeviceSynchronize();
        
        cudaEventRecord(start);
        for (int i = 0; i < 100; i++) {
            stridedAccess<<<gridSize, blockSize>>>(d_data, d_result, n, stride);
        }
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        
        float ms;
        cudaEventElapsedTime(&ms, start, stop);
        
        // Calculate effective bandwidth
        float dataBytes = 2.0f * numAccessed * sizeof(float) * 100;  // read + write
        float gbps = dataBytes / (ms * 1e6);
        float efficiency = 100.0f / stride;  // Simplified efficiency estimate
        
        printf("%-10d %-15.2f %-12.2f GB/s %-10.1f%%\n", stride, ms, gbps, efficiency);
    }
    printf("\n");
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_data);
    cudaFree(d_result);
}

void benchmarkCacheLevels() {
    printf("=== Exercise 2: L1 vs L2 Cache ===\n");
    
    const int dataSize = 1 << 24;
    const int iters = 1000;
    
    float *d_data, *d_result;
    CHECK_CUDA(cudaMalloc(&d_data, dataSize * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_result, 256 * sizeof(float)));
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Small working set (L1)
    int l1Size = 64;  // 64 floats = 256 bytes, fits in L1
    cudaEventRecord(start);
    l1CacheHit<<<1, 256>>>(d_data, d_result, l1Size, iters);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float l1Ms;
    cudaEventElapsedTime(&l1Ms, start, stop);
    
    // Large working set (L2/memory)
    cudaEventRecord(start);
    l2CacheMiss<<<1, 256>>>(d_data, d_result, dataSize, iters);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float l2Ms;
    cudaEventElapsedTime(&l2Ms, start, stop);
    
    printf("Small working set (L1): %.3f ms\n", l1Ms);
    printf("Large working set (L2): %.3f ms\n", l2Ms);
    printf("Ratio: %.2fx\n\n", l2Ms / l1Ms);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_data);
    cudaFree(d_result);
}

void benchmarkLdg() {
    printf("=== Exercise 3: __ldg Read-Only Cache ===\n");
    
    const int n = 1 << 24;
    
    float *d_data, *d_result;
    CHECK_CUDA(cudaMalloc(&d_data, n * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_result, n * sizeof(float)));
    
    int blockSize = 256;
    int gridSize = (n + blockSize - 1) / blockSize;
    
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Warmup
    normalLoad<<<gridSize, blockSize>>>(d_data, d_result, n);
    ldgLoad<<<gridSize, blockSize>>>(d_data, d_result, n);
    cudaDeviceSynchronize();
    
    // Benchmark normal
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        normalLoad<<<gridSize, blockSize>>>(d_data, d_result, n);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float normalMs;
    cudaEventElapsedTime(&normalMs, start, stop);
    
    // Benchmark __ldg
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        ldgLoad<<<gridSize, blockSize>>>(d_data, d_result, n);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float ldgMs;
    cudaEventElapsedTime(&ldgMs, start, stop);
    
    printf("Normal load: %.2f ms\n", normalMs);
    printf("__ldg load:  %.2f ms\n", ldgMs);
    printf("Speedup: %.2fx\n\n", normalMs / ldgMs);
    
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    cudaFree(d_data);
    cudaFree(d_result);
}

int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║              Cache Optimization Exercises                    ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n\n");
    
    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    printf("Device: %s\n", prop.name);
    printf("L2 Cache Size: %d KB\n\n", prop.l2CacheSize / 1024);
    
    benchmarkCoalescing();
    benchmarkCacheLevels();
    benchmarkLdg();
    
    printf("══════════════════════════════════════════════════════════════\n");
    printf("                    All exercises completed!\n");
    printf("══════════════════════════════════════════════════════════════\n");
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o cache_exercises cache_exercises.cu && ./cache_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: Cache Line Analysis

In [ ]:
# TODO: Calculate cache line utilization for different access patterns

def cache_line_efficiency(element_size, stride, warp_size=32, cache_line=128):
    """
    Calculate cache line efficiency.
    
    Args:
        element_size: Size of each element in bytes
        stride: Access stride (1 = consecutive)
        warp_size: Threads per warp
        cache_line: Cache line size in bytes
    
    Returns:
        Efficiency (0.0 - 1.0)
    """
    # TODO: Implement
    # Hint: Calculate how many cache lines are needed
    #       vs how much data is actually used
    pass

# Test
# cache_line_efficiency(4, 1)   # float, stride 1 → should be 100%
# cache_line_efficiency(4, 32)  # float, stride 32 → should be low

---

## Summary

### Cache Hierarchy

| Level | Size | Latency | Scope |
|-------|------|---------|-------|
| L1 | 48-128 KB | ~30 cycles | Per SM |
| L2 | 1.5-40 MB | ~200 cycles | Shared |
| Texture | ~48 KB | ~100 cycles | Per SM |
| Constant | 8 KB | ~5 cycles | Per SM |

### Coalescing Rules

1. **Consecutive threads → Consecutive addresses** (ideal)
2. **Cache line = 128 bytes**
3. **Stride-1 access = 100% efficiency**
4. **Stride-32+ = ~3% efficiency**

### CUDA C++ Patterns

```cpp
// Set cache preference
cudaFuncSetCacheConfig(kernel, cudaFuncCachePreferL1);

// Use read-only cache
float x = __ldg(&data[idx]);

// Constant memory
__constant__ float coefs[256];
cudaMemcpyToSymbol(coefs, h_coefs, sizeof(coefs));
```

### Tomorrow: Unified Memory
We'll explore simplified memory management with unified memory.